In [2]:
import imreg_dft as ird
import numpy
import video_processing as vp
import progressbar

In [ ]:
WIDTH, HEIGT = 256, 256
TRUE_FRAMERATE = 30

In [ ]:
class Trial:
    def __init__(self, filename):
        self.filename = filename
        self.correct_trials = []
        self.baselines = []
        
    def __str__(self, ):
        return(self.filename+str(self.correct_trials)+str(self.baselines))

### Determine which trials to use

In [ ]:
current_trial = None
good_trials = []

with open(mouse_text_file, 'r') as file:
    data = file.readlines()
    for line in data:
        line = line.replace('\n', '')
        line = line.split('\t')
        if line[0] == 'Movie':
            line[2] = line[2].replace(':', '/')
            if current_trial is not None:
                if len(current_trial.correct_trials) > 0:
                    good_trials.append(current_trial) 
            current_trial = Trial(hdd_directory+line[2])
        elif line[0] == 'stim':
            if "GO=2" in line[2]:
                current_trial.correct_trials.append(float(line[1]))
                current_trial.baselines.append(float(line[2][-4:]))
                
print(f"Number of good trials: {len(good_trials)}")            

### Translate the frames

In [ ]:
trial_frames = []
bottom_end = 100
top_end = 60
bool_have_template = False
#Threshold for which to tell if the mean changed due to lighting issues
threshold = 30
for trial in progressbar.ProgressBar(good_trials, max_value=len(good_trials)):
    frames = vp.extract_RAW_frames(trial.filename, WIDTH, HEIGHT,channel='green')
    for i, stim_end in enumerate(trial.correct_trials):
        stim_end = int(stim_end*TRUE_FRAMERATE)
        frames_of_trial = frames[stim_end-bottom_end:stim_end+top_end]
        mean_trial = numpy.mean(frames_of_trial, axis=(1,2))
        if (numpy.max(mean_trial)-numpy.min(mean_trial)) <= threshold:
            #do alignment of these images.
            if not bool_have_template:
                template_image = frames_of_trial[bottom_end]
                bool_have_template = True
            else:
                result = ird.translation(template_image, frames_of_trial[bottom_end])
                tvec = result['tvec']
                for i in range(len(frames_of_trial)):
                    frames_of_trial[i] = ird.transform_img(frames_of_trial[i], tvec=tvec)
            # Calculate baseline the -6 is to remove the 200ms of the stimulus
            baseline = numpy.mean(frames_of_trial[:bottom_end-6], axis=0)
            frames_of_trial = numpy.divide(
                numpy.subtract(frames_of_trial, baseline), 
                baseline
            )
            trial_frames.append(frames_of_trial)

trial_frames = numpy.asarray(trial_frames, dtype=numpy.float32) 

### Save the trial frames and the means of the frames

In [ ]:
mean_of_all_trials = numpy.mean(trial_frames, axis=0)
mean_of_all_trials.tofile(hdd_directory+mouse+"_average.raw")
trial_frames.tofile(hdd_directory+mouse+"_all_trials.raw")